import numpy as np

import requests
import datetime
from dateutil.relativedelta import relativedelta

# initialisation des dates de debut et fin d'extraction de données
date_debut = datetime.datetime.now() - relativedelta(days=2)
date_fin = date_debut + relativedelta(days=6)
# formatage des dates pour importer le bon URL
date_debut_formattee = date_debut.strftime('%Y-%m-%d') 
date_fin_formattee = date_fin.strftime('%Y-%m-%d')  

# copie de l'URL du site open-meteo avec les differents critères qui nous interessent
url = f"https://api.open-meteo.com/v1/meteofrance?latitude=43.6109&longitude=3.8763&hourly=temperature_2m,weathercode,precipitation&daily=weathercode,temperature_2m_max,temperature_2m_min,wind_speed_10m_max,precipitation_sum&timezone=Europe%2FLondon&start_date={date_debut_formattee}&end_date={date_fin_formattee}"

# récuperation des données
response = requests.get(url)
data = response.json()

# initialisation des tableaux de données
temp = data["hourly"]["temperature_2m"]
precipitation = data["daily"]["precipitation_sum"]
precipitation_h = data["hourly"]["precipitation"]
wind = data["daily"]["wind_speed_10m_max"]
tempmin = data["daily"]["temperature_2m_min"]
tempmax = data["daily"]["temperature_2m_max"]
icon = data["daily"]["weathercode"]

def date(i):
    
    d = datetime.datetime.now() + relativedelta(days=i)
    return f"{d.day}-{d.month}-{d.year}"

def index(i):
    
    jours_semaine = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return jours_semaine[(datetime.datetime.now().weekday() + i) % 7]  
def icon(i):
    """
    Association de chaque code WBO à un pictogramme correspondant. Retourne le chemin d'accès au pictogramme par une chaîne de caractères. Version daily.
    """
    code = icon[i]

    if code == 0:
        return "icon/wi-day-sunny.svg"
    elif code == 1:
        return "icon/wi-day-sunny-overcast.svg"
    elif code == 2:
        return "icon/wi-day-cloudy.svg"
    elif code == 3:
        return "icon/wi-cloud.svg"
    elif code in [45, 48]:
        return "icon/wi-fog.svg"
    elif code in [51, 53, 55, 56, 57]:
        return "icon/wi-sprinkle.svg"
    elif code in [61, 81, 82]:
        return "icon/wi-showers.svg"
    elif code == 63:
        return "icon/wi-rain-wind.svg"
    elif code in [65, 66, 67]:
        return "icon/wi-rain.svg"
    elif code in [71, 73, 75, 85, 86]:
        return "icon/wi-snow.svg"
    elif code in [95, 96]:
        return "icon/wi-storm-showers.svg"
    elif code == 99:
        return "icon/wi-thunderstorm.svg"
    else:
        return "icon/wi-moon-new.svg"

def iconrain(i):
    
    """
    Retourne une image sous format HTML correspondant à l'intensité des précipitations.
    """
    if not isinstance(precipitation[i], float):
        return ""

    intensity = precipitation[i]

    if 0 < intensity <= 2:
        return f'<img src="icon/wi-raindrop.svg" width="20" height="20" />'
    elif intensity > 2:
        return f'<img src="icon/wi-raindrops.svg" width="40" height="40" />'
    else:
        return ""  
# Calcule la somme des 24 dernières valeurs de precipitation_h
precipitation_h = [0 if value is None else value for value in precipitation_h]
precipitationd = sum(precipitation_h[-24:])        